# Q.1 Theory and concepts 

a. Batch normalization is a technique used to improve the training of deep neural networks by normalizing the inputs of each layer. This is done by standardizing the inputs to have a mean of zero and a variance of one within each mini-batch. Batch normalization helps to mitigate issues related to internal covariate shift, where the distribution of inputs to a layer changes during training.



b. Stabilizes Learning: Normalizing inputs reduces the variation in the distribution of activations, leading to more stable and faster convergence.
Higher Learning Rates: Allows the use of higher learning rates, which can speed up the training process.
Regularization Effect: Acts as a form of regularization, reducing the need for other regularization techniques such as dropout.
Improved Gradient Flow: Helps in maintaining gradients within a reasonable range, reducing issues related to vanishing/exploding gradients.
Reduces Sensitivity to Initialization: Makes the network less sensitive to the initial choice of parameters. 

# Q.2 implementation-

In [1]:
!pip install tensorflow
!pip install keras 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 2.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 59.2 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 61.2 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 59.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 41.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 64.1 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133

In [ ]:


# import require libraries
import tensorflow as tf 
from tensorflow.keras import layers,models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical


#load dataset 

(x_train,y_train),(x_test,y_test) = cifar10.load_data()

# Normalize Pixel
x_train,x_test = x_train/255.,x_test/255.

# One Hot encodding 
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

#  Define the neural network
model = models.Sequential([
    layers.Flatten(input_shape=(32,32,3)),
    layers.Dense(512,activation='relu'),
    layers.Dense(10,activation='softmax')

])

model.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics=['accuracy']

)

history_no_bn = model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

model_bn = models.Sequential([
    layers.Flatten(input_shape=(32,32,3)),
    layers.Dense(512),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dense(10,activation='softmax')

])

model_bn.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics=['accuracy']

)

history_bn = model_bn.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

import matplotlib.pyplot as plt 

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history_no_bn.history['accuracy'],label='train (no bn)')
plt.plot(history_no_bn.history['val_accuracy'],label='validation no bn')
plt.plot(history_bn.history['accuracy'],label='train ( bn)')
plt.plot(history_bn.history['val_accuracy'],label='validation bn')
plt.title('model training')
plt.xlabel('train')
plt.ylabel('epochs')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history_no_bn.history['loss'],label='train (no bn)')
plt.plot(history_no_bn.history['val_loss'],label='validation no bn')
plt.plot(history_bn.history['loss'],label='train ( bn)')
plt.plot(history_bn.history['val_loss'],label='validation bn')
plt.title('model loss')
plt.xlabel('loss')
plt.ylabel('epochs')
plt.legend()
plt.tight_layout()
plt.show()

batch_sizes = [32,64,128,256]
result = {}

for batch_size in batch_sizes:
    print(f'traing for batch size {batch_size}')
    model_bn = models.Sequential([
        layers.Flatten(input_shape=(32,32,3)),
        layers.Dense(512),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dense(10,activation='softmax')

    ])

    model_bn.compile(
        optimizer='adam',
        loss = 'categorical_crossentropy',
        metrics=['accuracy']

    )

    history = model_bn.fit(x_train,y_train,epochs=10,batch_size=batch_size,validation_data=(x_test,y_test))

    result[batch_size] = history



plt.figure(figsize=(14,10))
for i,batch_size in enumerate(batch_sizes):
  history = result[batch_size]
  plt.subplot(2,2,i+1)
  plt.plot(history.history['accuracy'],label='training accuracy')
  plt.plot(history.history['val_accuracy'],label='validation accuracy')
  plt.title(f'batch size {batch_size}')
  plt.xlabel('train')
  plt.ylabel('epoches')
  plt.legend()

plt.tight_layout()
plt.show()



# Q.3 

Advantages:

Accelerated Training: Batch normalization allows the use of higher learning rates and leads to faster convergence.
Improved Generalization: Acts as a form of regularization, helping to prevent overfitting.
Stabilized Training: Reduces sensitivity to initialization and learning rate choices, leading to more stable training dynamics.
Gradient Flow: Mitigates issues related to vanishing and exploding gradients, particularly in deep networks.
Potential Limitations:

Dependency on Batch Size: The performance of batch normalization can be sensitive to the choice of batch size.
Computational Overhead: Adds some computational overhead due to the additional normalization and parameter scaling operations.
Not Always Effective: In certain cases, other normalization techniques such as layer normalization or instance normalization may be more effective, especially for recurrent neural networks or small batch sizes.
Batch Statistics: Inconsistent batch statistics during training and inference (batch vs. running mean/variance) can lead to discrepancies and require careful handling.